In [3]:
import numpy as np
from random import shuffle
from sys import exit
from copy import deepcopy

In [4]:
card_per_player = {2:7, 3:6, 4:6, 5:6}
players = 2
card_per_turn = 2
danger_thresh = 10
one_more_tresh = 2

N = players
n = 2 #card_per_turn
m = card_per_player[players]

In [5]:
def pick_one(hand):
    global n,deck_empty
    if deck == [] and not deck_empty:
        n = card_per_turn - 1
        print("\n------------------------------- Deck is over, last few rounds. Hold on! -------------------------------\n")
        deck_empty = True
    if deck == []:
        return hand
    hand.append(deck[0])
    deck.pop(0)
    return hand

In [6]:
def scan_hand_(hand,discards=[],obligatory_move=True):
# delta is the distance of the bast match
# ind contains first the in hand index then the pile index
    tops = [pile[-1] for pile in piles]
    delta = 100
    ind = []
    for i, hand_i in enumerate(hand):
        for j, top_j in enumerate(tops):
            if j in discards: continue
            old_delta = delta
            old_ind = ind
            delta = hand_i - top_j
            if j > 1: delta = -delta
            ind = [i,j]
            if delta == -10:
                return delta, ind
            elif delta >= old_delta or delta < 0 :
                delta = old_delta
                ind = old_ind
    #print("Current status:", delta, discards, obligatory_move)
    if delta==100 and discards==[] and obligatory_move: end_game_and_recap() #replace this with exit program command when the code is implemented in a script
    return delta, ind

In [7]:
def scan_hand(hand,discards=[],obligatory_move=True):
    delta,ind = scan_hand_(hand,discards=discards,obligatory_move=obligatory_move)
    if ind==[]:
        return delta,[None,None]
    elif len(ind)==2:
        return delta,ind
    else:
        raise ValueError("Output of scan_hand not recognized")

In [8]:
def print_piles_vertical(piles):
    # Find the maximum length among all lists
    max_len = max(len(pile) for pile in piles)
    
    # Print the piles in columns
    for i in range(max_len):
        row = []
        for pile in piles:
            # If the pile still has elements at this index, add the element
            if i < len(pile):
                row.append(str(pile[i]))
            else:
                row.append("")  # Empty space if the pile is finished
        
        # Print the row with uniform spacing
        print(f"{row[0]:<4} {row[1]:<4} {row[2]:<4} {row[3]:<4}")

In [9]:
def print_piles(piles):

    print("")
    print("Printing Piles")
    for pile in piles:
        string = ""
        for element in pile:
             string = string + f"{element:3d}" + " "
        print(string)
    print("")

In [10]:
def end_game_and_recap():
    #print_piles(piles)
    raise ValueError("Game Over")

In [31]:
#interaction(hands[inactive_players])
def interaction(hands):
    print("")
    print("Interaction")
    #things to implement: give higher priority to closest players
    n_hands = len(hands)
    delta_list = []
    pile_idx_list = []
    for i in range(n_hands):
        delta,(card_idx,pile_idx)=scan_hand(hands[i],obligatory_move=False)
        pile_idx_list.append(pile_idx)
        if card_idx is not None: print("Card ",hands[i][card_idx]," playable on pile ",pile_idx,"with delta ",delta)
    print("")
    return list(set(pile_idx_list))

In [12]:
def place_card(hand,card_idx,pile_idx):
    print('Card placed: ', hand[card_idx], 'on pile ',pile_idx)
    piles[pile_idx].append(hand[card_idx]) #add played card to the piles
    hand.pop(card_idx) #remove card played from the hand
    return hand

In [13]:
def play_card(hand,player_idx,obligatory_move=True):
    delta_1,(card_idx1,pile_idx1)=scan_hand(hand,obligatory_move=obligatory_move)
    if delta_1==-10:
        hand=place_card(hand,card_idx1,pile_idx1)
    elif not obligatory_move and delta_1>one_more_tresh:
        return hand
    else:
        #mask_inactive = [i!=player_idx for i in range(N)]
        hands_non_active = [hand for i,hand in enumerate(hands) if not i==player_idx] #np.asarray(hands)[mask_inactive].tolist()
        dont_pile_idx_list = interaction(hands_non_active)
        
        if not pile_idx1 in dont_pile_idx_list: #case 1: the pile of interest can be played
            hand=place_card(hand,card_idx1,pile_idx1)
        elif dont_pile_idx_list==[0,1,2,3]: #case 2: all piles are taken, so fuck it
            hand=place_card(hand,card_idx1,pile_idx1)
        else: #case 3: check other piles
            delta_2,(card_idx2,pile_idx2)=scan_hand(hand,discards=dont_pile_idx_list,obligatory_move=obligatory_move)
            if delta_2-delta_1>danger_thresh or delta_2==100:
                hand=place_card(hand,card_idx1,pile_idx1)
            else:
                if not obligatory_move and delta_2>one_more_tresh:
                    return hand
                else: hand=place_card(hand,card_idx2,pile_idx2)
    return hand    

In [14]:
def play_turn(player_idx):
    n_played=n   #this is incremented only in case one or more non-obbligatory cards are played
    hand=hands[player_idx]

    print('player ', player_idx,' , hand: ',hand)
    print_piles(piles)
    
    for _ in range(n):
        hand=play_card(hand,player_idx)

    while True:
        hand_old=hand.copy()
        hand=play_card(hand,player_idx,obligatory_move=False)
        #print(hand,hand_old)
        if hand==hand_old: break
        n_played = n_played + 1
        
    print('n. played cards: ',n_played,'\n')
    for _ in range(n_played):
        hand=pick_one(hand)
    hands[player_idx]=hand

In [29]:
original_deck = [int(i) for i in np.arange(2,99)]
shuffle(original_deck)
original_piles = [[1],[1],[100],[100]]

In [32]:
#Convention: decreasing piles start with 100 but for sake of ease
# we consider them at -100 so that they are increasing
# in this way we simply change the sign for cards in hands
deck = deepcopy(original_deck)
piles = deepcopy(original_piles)
hands = np.asarray(deck[:N*m]).reshape([N,m]).tolist()
deck = deck[N*m:]

deck_empty = False
n = 2 #card_per_turn

while True:
    for player_idx in range(N):
        play_turn(player_idx)
        if hands==[[]*N]: break
print('Win')

player  0  , hand:  [88, 12, 50, 34, 27, 47, 65]

Printing Piles
  1 
  1 
100 
100 


Interaction
Card  95  playable on pile  2 with delta  5

Card placed:  12 on pile  0

Interaction
Card  95  playable on pile  2 with delta  5

Card placed:  88 on pile  3
n. played cards:  2 

player  1  , hand:  [43, 83, 70, 95, 24, 77, 69]

Printing Piles
  1  12 
  1 
100 
100  88 


Interaction
Card  27  playable on pile  0 with delta  15

Card placed:  83 on pile  3

Interaction
Card  27  playable on pile  0 with delta  15

Card placed:  95 on pile  2
n. played cards:  2 

player  0  , hand:  [50, 34, 27, 47, 65, 46, 54]

Printing Piles
  1  12 
  1 
100  95 
100  88  83 


Interaction
Card  77  playable on pile  3 with delta  6

Card placed:  27 on pile  0

Interaction
Card  77  playable on pile  3 with delta  6

Card placed:  34 on pile  0
n. played cards:  2 

player  1  , hand:  [43, 70, 24, 77, 69, 62, 23]

Printing Piles
  1  12  27  34 
  1 
100  95 
100  88  83 

Card placed:  24 on pile

ValueError: Game Over

In [158]:
## cell test